In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy
from astropy.io import fits
from matplotlib import rcParams
from astropy.wcs import WCS 
from pathlib import Path
rcParams['figure.figsize'] = [12., 12.]


# Read in a fits file to plot

In [ ]:
fits_file= "d100.fits"

with fits.open(fits_file) as hdul: 
    #create 2d array in galactic coordinates
    data = hdul[0].data
    header = hdul[0].header
    

# Graph with a log scale to see better

In [ ]:

plt.imshow(np.log10(data), origin = 'lower', cmap = 'cividis', aspect= 'equal')
plt.savefig("Bias_Image.pdf")

In [ ]:
#sanity check for histogram
print(type(data.flatten()))
print(data.flatten().shape)

# Plot the flattened 2d array

In [ ]:
plt.hist(data.flatten(),bins=50)
plt.show()

# I'm assuming tthe signal is creating an outlier value so if we log scale we may see the distribution better

In [ ]:
plt.hist(np.log10(data.flatten()))
plt.savefig("Count_Distribution.pdf")
# plt.show()

# I looked up a read in method in astropy and a visualization technique to remove outliers (I realize I am removing 2% of the data)

In [ ]:
image_data = fits.getdata(fits_file, est=0)
print(image_data.flatten().shape)
image_data = image_data[~np.isnan(image_data)]

# removing the outliers at each end
core = image_data[(image_data > np.percentile(image_data, 1)) & 
                     (image_data < np.percentile(image_data, 99))]
histogram = plt.hist(core, bins='auto')
plt.savefig("Count_Distribution.pdf")

In [ ]:
#Alternative method to flatten. Proabobly better
# data_ravel = np.ravel(data)
# np.shape(data_ravel)

In [ ]:
# plt.hist(data_ravel.flatten(), bins=50)

# Read in files from the folder instead of one at a time (if not located in same jupyter partition then you need to use the file path to the folders location) and then sort the bias and flats

In [ ]:

directory = Path('.')


fits_files = [f for f in directory.iterdir() if f.suffix.lower() == '.fits']
image_data = []
image_headers = []
records=[]

for f in fits_files:
    with fits.open(f) as hdul:
        image_data.append(hdul[0].data)
        image_headers.append(hdul[0].header)

bias_data = [
    data for f, data in zip(fits_files, image_data)
    if f.stem.startswith('d') and 100 <= int(f.stem.split('d')[-1]) <= 109
]

flats_data = [
    data for f, data in zip(fits_files, image_data)
    if f.stem.startswith('d') and 125 <= int(f.stem.split('d')[-1]) <= 140
]


# Plot bias mean

In [ ]:
# stack bias into a 3d array, then take meadian (arg1,...) axis 0  to compute median pixel value
plt.imshow(np.log10(np.median(np.stack(bias_data),axis=0)), origin = 'lower', cmap = 'cividis', aspect= 'equal')
plt.savefig("bias_mean.pdf")

# Plot flat mean same way 

In [ ]:
plt.imshow(np.log10(np.median(np.stack(flats_data),axis=0)), origin = 'lower', cmap = 'cividis', aspect= 'equal')
plt.savefig("Flat_mean.pdf")

# Subtraction Clean Flat

In [ ]:

master_bias=np.median(np.stack(bias_data),axis=0)
master_flat=np.median(np.stack(flats_data),axis=0)

clean_flat=master_flat-master_bias


# Normalize the Flats

In [ ]:
normalized_flat= clean_flat / np.mean(master_flat)
#normalized_flat = np.clip(normalized_flat, 0, None)  # set negative values to 0
plt.imshow(np.log10(normalized_flat), origin = 'lower', cmap = 'cividis', aspect= 'equal')
plt.savefig("Normalized_Flats.pdf")

# Science Image

In [ ]:
science_data = []
science_headers = []
for f, data, header in zip(fits_files, image_data, image_headers):
    if f.stem.startswith('d') and 219 <= int(f.stem.split('d')[-1]) <= 227:
        science_data.append(data)
        science_headers.append(header)
    

In [ ]:
clean_science=[]
for i in science_data:
    #iterate through science images and append the operation to clean science
    clean_science.append(i-master_bias)
persec_science = []
#this for loop pairs up iterations of both data frames, anything unpaired is ignored
for img, header in zip(clean_science, science_headers):
    #call exposure time
    exptime = header.get('EXPTIME', 1) 
    #returns 1 if nan or 0  
    #divide exposure time
    persec_science.append(img / exptime)
#take the median of persec
master_science=np.median(np.stack(persec_science),axis=0)

calibrated_scienceimage=[]
#create calibrated by dividing master science by normalized flat
calibrated_scienceimage=(master_science/normalized_flat)

plt.imshow(np.log10(calibrated_scienceimage), origin = 'lower', cmap = 'cividis', aspect= 'equal')
plt.savefig("Calibrated_Science_Image.pdf")

# Just plotted one science image

In [ ]:
plt.imshow(np.log10(i/normalized_flat), origin = 'lower', cmap = 'cividis', aspect= 'equal')